In [ ]:
ROOTPATH = '/Users/zhenggong/Documents/Github/IBES_selected/capitaliq' # for importing and reference management 
import sys
sys.path.append(ROOTPATH)
import pandas as pd 
from cfg import SERVER_TIMEZONE,DBINFO
import json
import psycopg2
from datetime import datetime, timedelta


def read_sql_to_df(sql, db, cursor):
    """Internal: execute sql and get dataframe as a return 
    
    Args:
        sql (str): sql to be executed 
        db (psycopg2.connect): connect to ciq target database 
        cursor (database connection.cursor): 
    
    Returns:
        pd.DataFrame: contains result for executed sql 
    """
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(data,columns = columns)
    except (Exception, psycopg2.DatabaseError) as error:
        db.rollback()
        print("Error: %s" % error)


def get_connection(dbInfo):
    """get the db connection using info in dbInfo json file
    
    Args:
        dbInfo (.json): user-pw for ciq target database 
    
    Returns:
        psycopg2.connect
    """
    with open(dbInfo,'r') as f:
        u = json.load(f)

    db = psycopg2.connect(
        host=u['host'],
        database=u["database"],
        user=u["user"],
        password=u["pwd"])
    return db

###################################################################################
def get_transcriptperson(connection = None):


    sql = f"""
    SELECT * from ciqtranscriptperson;
    """
    if connection is None:
        connection = get_connection(DBINFO)
    cursor = connection.cursor()
    return read_sql_to_df(sql, connection, cursor)
    

res = get_transcriptperson()
res.to_csv('transcriptperson.csv',index = False)
